# Read Data and Fill Missing Values

In [1]:
import pandas as pd

data = pd.read_csv('../data/agaricus-lepiota.data', sep=",", header=None)
random
for col in data.columns:
        mod = data[col].mode()[0]
        data[col] = data[col].replace('?', mod)

# Convert Nominal Data To Numeric

In [2]:
obj_cols = data.select_dtypes(['object']).columns
data[obj_cols] = data[obj_cols].astype('category')
cat_cols = data.select_dtypes(['category']).columns
data[cat_cols] = data[cat_cols].apply(lambda x: x.cat.codes)

# Holdout

In [3]:
import math
import random

test_set_size = math.floor(len(data.index) / 3)
test_set_indexes = random.sample(range(0,len(data.index)), test_set_size)
test_set = pd.DataFrame(data, index=test_set_indexes)
train_set = data.drop(test_set_indexes)

# Run KNN

In [4]:
from sklearn.neighbors import KNeighborsClassifier

training_set_data = (pd.DataFrame(train_set, columns=range(1, len(train_set.columns)))).values.tolist()
training_set_result = (pd.DataFrame(train_set, columns=[0])).values.tolist()
# training_set_result reformat
tsr_reformat = []
for i in range(0, len(training_set_result)):
    tsr_reformat.append(training_set_result[i][0])
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(training_set_data, tsr_reformat)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

# Test Model

In [5]:
from sklearn.metrics import precision_recall_fscore_support

testing_set_data = (pd.DataFrame(test_set, columns=range(1, len(test_set.columns)))).values.tolist()
testing_set_result = (pd.DataFrame(test_set, columns=[0])).values.tolist()
predicted = knn.predict(testing_set_data)
precision, recall, fscore = (precision_recall_fscore_support(testing_set_result, predicted, beta=1, average='binary'))[0:3]
print("Precision = ", precision)
print("Recall = ", recall)
print("fscore(beta=1) = ", fscore)

Precision =  1.0
Recall =  0.9985107967237528
fscore(beta=1) =  0.9992548435171386
